<h1 "style = color:blue"> Chapter 4 </h1>

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

In [ ]:
cnx = pd.read_csv("Exploratory.csv")

In [ ]:
cnx.head()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
cnx['Total income'].describe()

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(8,4))

sns.boxplot(y = 'Total income',data=cnx,ax=axes[0])
axes[0].set_title("Histogram")
sns.kdeplot(data=cnx['Total income'],ax = axes[1])
axes[1].set_title("Density Plot")
fig.tight_layout(pad=3.0);

In [ ]:
sm.graphics.qqplot(cnx['Total income']);

In [ ]:
scipy.stats.shapiro(cnx['Total income'].dropna())

In [ ]:
cnx['Industry type'].value_counts()

In [ ]:
cnx['Industry type'].value_counts().plot(kind='bar');

In [ ]:
cnx['Industry type'].astype('category')

In [ ]:
cnx['Industry type'].replace(to_replace=[1,2,3],
                             value=['Non-Finance','NBFC',"Banks"]).value_counts().plot(kind='bar');

<h2 "style=color:red"> Multivariate Analysis </h2>

In [ ]:
cnx[['Total income','Market Capitalisation']].corr()

In [ ]:
bcnx.corr()['Market Capitalisation'].sort_values(ascending=False)

In [ ]:
cnx = sm.add_constant(cnx)

In [ ]:
cnx.head(2)

In [ ]:
cnx1 = cnx[['const','Total income','Market Capitalisation']].dropna()

In [ ]:
results = sm.OLS(cnx1['Market Capitalisation'],cnx1[['const','Total income']]).fit()

In [ ]:
results.summary()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,6))
sns.regplot(x='Total income',y='Market Capitalisation',data=cnx,lowess = True,ax=ax[0])
sns.regplot(x='Total income',y='Market Capitalisation',data=cnx,ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
fig = sm.graphics.influence_plot(results, alpha  = 0.05, ax = ax, criterion="cooks");

In [ ]:
cnx['margin']=cnx['Profit after tax']/cnx['Total income']

In [ ]:
sns.boxplot(x='Industry type',y='margin',data=cnx);

In [ ]:
cnx['Ownership']=np.where(cnx['Ownership group'].str.contains("(F)"),"MNC",
                         np.where(cnx['Ownership group'].str.contains("Group"),"BusGrp",
                         np.where(cnx['Ownership group'].str.contains("Govt"),"PSU","Others")))
cnx.head(2)

In [ ]:
pd.crosstab(cnx['Ownership'],cnx['Industry type'])

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

In [ ]:
mosaic(cnx,['Ownership',"Industry type"]);

In [ ]:
sns.heatmap(pd.crosstab(cnx['Ownership'],cnx['Industry type']),cmap="Blues");

In [ ]:
stat, p, df, exp = scipy.stats.chi2_contingency(pd.crosstab(cnx['Ownership'],cnx['Industry type']))

In [ ]:
p

<h2 "style=color:red"> Analysis of TimeSeries Data </h2>

In [ ]:
nifty = yf.download("^NSEI",progress=False)
nifty.iloc[[0,nifty.shape[0]-1],:]

In [ ]:
nifty.Close.plot()
plt.title("Nifty over 2007 - 2021");

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
print(f"ADF Statistic = {adfuller(nifty.Close)[0]}")
print(f"p_value = {adfuller(nifty.Close)[1]}")

In [ ]:
nifty['return_dis']=nifty.Close.pct_change()
nifty['return_cont']=np.log(nifty.Close.pct_change().add(1))
nifty = nifty.dropna()
nifty.head(2)

In [ ]:
sns.kdeplot(nifty.return_cont);

In [ ]:
nifty.loc[[nifty.return_cont.idxmax()]]

In [ ]:
nifty.loc[[nifty.return_cont.idxmin()]]

In [ ]:
from scipy.stats import jarque_bera

In [ ]:
jarque_bera(nifty.return_cont)

In [ ]:
print(f"ADF Statistic = {adfuller(nifty.return_cont)[0]}")
print(f"p_value = {adfuller(nifty.return_cont)[1]}")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plot_acf(nifty.return_cont,zero=False)

In [ ]:
nifty['dow']=nifty.index.dayofweek
# Monday is 0
nifty.groupby('dow').return_cont.mean().plot(kind='bar');

In [ ]:
nifty_monthly = nifty.resample('M').last()
nifty_monthly['return_cont']=np.log(nifty_monthly.Close.pct_change().add(1))
nifty_monthly[['Close','return_cont']].dropna().head(3)

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(nifty.return_cont,lags=5,return_df=True)